In [1]:
import torch
from transformers import  get_linear_schedule_with_warmup # for training
from torch.cuda.amp import autocast
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
import os
from datasets import load_dataset

/home/service/miniconda3/envs/llm_research/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb
wandb.init(project="biollama_ft", # the project I am working on
           tags=["hf_sft", "Llama2"]) # the Hyperparameters I want to keep track of
# artifact = wandb.use_artifact('Neelectric/MedQA-USMLE', type='dataset')
# artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nelectric (neelectric). Use `wandb login --relogin` to force relogin


In [3]:
from utilities.parse_benchmark import parse_benchmark

benchmark = "MedQA"
artifact_dir = os.getcwd() + "/benchmarks/MedQA-USMLE/"
#dataset = load_dataset("Neelectric/MedQA-USMLE")
medqa = load_dataset("json", data_dir=artifact_dir)


In [4]:
train_dataset = medqa["train"]
eval_dataset = medqa["validation"]
#print sizes
print(len(train_dataset))
print(len(eval_dataset))

10178
1272


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

################################################################################
# bitsandbytes parameters
################################################################################



# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

bf16 = False
fp16 = True
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
        bf16 = True
        fp16 = False


Your GPU supports bfloat16: accelerate training with bf16=True


In [6]:
# Load the entire model on the GPU 0
# device_map = {"": 0} # lets load on the next
# device = torch.device('cuda:0')

# Load base model
if bf16:
    torch_dtype=torch.bfloat16
else:
    torch_dtype=torch.float16

# This works, this is training the qunatised model
model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    torch_dtype=torch_dtype,
    quantization_config=bnb_config,
    device_map="auto"
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.91s/it]


In [10]:

# Accessing layers and weights

for param_name, param in model.named_parameters():
    print(f"\tWeight: {param_name} | Size: {param.size()} | Trainable: {param.requires_grad}")  # Print first 2 values of each weight for brevity

	Weight: model.embed_tokens.weight | Size: torch.Size([32000, 4096]) | Trainable: True
	Weight: model.layers.0.self_attn.q_proj.weight | Size: torch.Size([8388608, 1]) | Trainable: False
	Weight: model.layers.0.self_attn.k_proj.weight | Size: torch.Size([8388608, 1]) | Trainable: False
	Weight: model.layers.0.self_attn.v_proj.weight | Size: torch.Size([8388608, 1]) | Trainable: False
	Weight: model.layers.0.self_attn.o_proj.weight | Size: torch.Size([8388608, 1]) | Trainable: False
	Weight: model.layers.0.mlp.gate_proj.weight | Size: torch.Size([22544384, 1]) | Trainable: False
	Weight: model.layers.0.mlp.up_proj.weight | Size: torch.Size([22544384, 1]) | Trainable: False
	Weight: model.layers.0.mlp.down_proj.weight | Size: torch.Size([22544384, 1]) | Trainable: False
	Weight: model.layers.0.input_layernorm.weight | Size: torch.Size([4096]) | Trainable: True
	Weight: model.layers.0.post_attention_layernorm.weight | Size: torch.Size([4096]) | Trainable: True
	Weight: model.layers.1.self